# Mars CTX Stereo Tool


# Imports

In [151]:
from shapely.geometry import shape

In [190]:
import geopandas as gp
from sidecar import Sidecar
import os
import json
from ipyleaflet import TileLayer, Map, DrawControl, SearchControl, Marker, AwesomeIcon, GeoJSON, GeoData, LayerGroup 
from ipyleaflet import WidgetControl, GeoJSON, basemap_to_tiles, LayersControl, projections, FullScreenControl
from ipywidgets import Text, HTML


# Defs

In [153]:
mars_eqc_crs = {'custom': True, 'name': 'Mars2000', 'proj4def': '+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +a=3396190 +b=3396190 +units=m +no_defs', }
mars_np_crs = {'custom': True, 'name': 'IAU2000:49919', 'proj4def': '+proj=stere +lat_0=90 +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=3396190 +b=3376200 +units=m +no_defs', }
mars_sp_crs = {'custom': True, 'name': 'IAU2000:49920', 'proj4def': '+proj=stere +lat_0=-90 +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=3396190 +b=3376200 +units=m +no_defs', }

In [154]:
ctx_fp_url = 'https://ode.rsl.wustl.edu/mars/datafile/derived_products/coverageshapefiles/mars/mro/ctx/edr/mars_mro_ctx_edr_c0a.zip'

In [155]:
ctx_df = gp.read_file(ctx_fp_url)

In [156]:
good_columns = ['ProductId','Ext2URL', 'EmAngle', 'InAngle', 'PhAngle']

In [200]:
mola_color_shade_url = 'https://astro.arcgis.com/arcgis/rest/services/OnMars/MColorDEM/MapServer/tile/{z}/{y}/{x}'
mola_color_bm = {
    'url': mola_color_shade_url,
    'attribution': 'USGS/ESRI/NASA',
    'crs':projections.EPSG4326,
    'max_native_zoom': 5,
}

In [202]:
def perform_stereo_query(df=ctx_df):
    # get query as shape
    bbox = shape(querys['features'][0]['geometry'])
    minx, miny, maxx, maxy = bbox.bounds
    # filter df down to everything intersecting the bbox
    df_fp1 = df.cx[minx:maxx, miny:maxy]
    # for now don't allow more than 100 ctx footprints, replace with something smarter later
    if len(df_fp1) > 100:
        df_fp1 = df_fp1.iloc[0:100]
    # perform more of query
    left = df_fp1.query('EmAngle < 5.0').copy()
    left.loc[:,'lg'] = left['geometry']
    right = df_fp1.query('EmAngle >= 5.0').copy()
    right.loc[:,'rg'] = right['geometry']
    # todo: drop any duplicates which shouldn't be possible but floats are bad
    df_fp2 = gp.overlay(left, right , how='intersection')
    df_fp2['diff_em'] = (df_fp2['EmAngle_2'] - df_fp2['EmAngle_1']).abs() 
    df_fp2['ovarea'] = df_fp2['geometry'].to_crs(mars_eqc_crs['proj4def']).area
    #
    # tt['ovarea_wl'] = gp.overlay(tt['geometry'], tt['lg'], how='intersection').area / tt['lg'].area * 100
    # tt['ovarea_wr'] = gp.overlay(tt['geometry'], tt['rg'], how='intersection').area / tt['rg'].area * 100
    # done!
    del df_fp2['lg']
    del df_fp2['rg']
    return df_fp2
    

In [203]:
good_query_columns = ['ProductId_1', 'ProductId_2', 'EmAngle_1', 'EmAngle_2', 'diff_em', 'ovarea', 'Ext2URL_1', 'Ext2URL_2',]

In [204]:
querys = {
    'type': 'FeatureCollection',
    'features': [],
    'results': []
}
# hover widget 
html = HTML('''Hover over a state''')
html.layout.margin = '0px 20px 20px 20px'
hovercontrol = WidgetControl(widget=html, position='topright')
# map
m = Map(
    center=(0,0),
    zoom=1, 
    basemap=TileLayer(**mola_color_bm),
    crs=projections.EPSG4326,
)

control = FullScreenControl()
draw_control = DrawControl()
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": .1
    }
}

def on_selection(self, action, geo_json):
    if len(querys['features'])>=1:
        querys['features'] = list()
    querys['features'].append(geo_json)
    # perform query
    dfq = perform_stereo_query()
    querys['results'].append(dfq)
    # show df
    qgrid.show_grid(dfq[good_columns])
    # update map
    gd = GeoData(geo_dataframe=dfq[['geometry']])
    m.add_layer(gd)
    

draw_control.on_draw(on_selection)
m.add_control(hovercontrol)
m.add_control(control)
m.add_control(draw_control)
sc = Sidecar(title='Mars')
with sc:
    display(m)

In [188]:
gd = GeoData(geo_dataframe=querys['results'][-1].sort_values('ovarea', ascending=False),
            hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},)
m.add_layer(gd)

IndexError: list index out of range

In [162]:
def update_html(feature,  **kwargs):
    html.value = '''
        <h3><b>{}</b></h3>
        <a target="_blank" rel="noopener noreferrer" href="{}"><h4>LeftImg: {}</h4></a>
        <a target="_blank" rel="noopener noreferrer" href="{}"><h4>RightImg: {}</h4></a>
    '''.format(feature['properties']['diff_em'],
               feature['properties']['Ext2URL_1'],
               feature['properties']['ProductId_1'],
               feature['properties']['Ext2URL_2'],
               feature['properties']['ProductId_2'])

gd.on_hover(update_html)



In [139]:
querys['results'][-1][good_query_columns]

,ProductId_1,ProductId_2,EmAngle_1,EmAngle_2,diff_em,ovarea,Ext2URL_1,Ext2URL_2
0,P02_002007_1796_XN_00S345W,P02_001796_1799_XI_00S345W,0.09,7.50,7.41,1.176710e+09,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
1,F01_036238_1787_XI_01S345W,P02_001796_1799_XI_00S345W,4.25,7.50,3.25,9.244243e+08,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
2,F04_037372_1798_XN_00S345W,P02_001796_1799_XI_00S345W,1.91,7.50,5.59,1.180804e+09,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
3,F05_037794_1798_XN_00S345W,P02_001796_1799_XI_00S345W,2.30,7.50,5.20,1.190912e+09,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
4,P02_002007_1796_XN_00S345W,P03_002218_1800_XI_00S345W,0.09,8.79,8.70,1.234248e+09,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
5,F01_036238_1787_XI_01S345W,P03_002218_1800_XI_00S345W,4.25,8.79,4.54,8.649319e+08,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
6,F04_037372_1798_XN_00S345W,P03_002218_1800_XI_00S345W,1.91,8.79,6.88,1.214962e+09,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
7,F05_037794_1798_XN_00S345W,P03_002218_1800_XI_00S345W,2.30,8.79,6.49,1.225128e+09,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
8,P02_002007_1796_XN_00S345W,P03_002363_1797_XI_00S345W,0.09,8.48,8.39,1.765137e+09,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...
9,F01_036238_1787_XI_01S345W,P03_002363_1797_XI_00S345W,4.25,8.48,4.23,1.994773e+09,http://viewer.mars.asu.edu/planetview/inst/ctx...,http://viewer.mars.asu.edu/planetview/inst/ctx...


In [134]:
qgrid.show_grid(querys['results'][-1][good_query_columns])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [112]:
import json

In [117]:
gd.data=json.loads(querys['results'][-1].to_json()) # how to get the layer to update

In [ ]:
query_string = 
"""
SELECT 
 a.ProductId as left_pi, 
 b.ProductId as right_pi,
 a.EmAngle as left_em,
 b.EmAngle as right_em,
 a.Ext2URL as left_url, 
 b.Ext2URL as right_url, 
 ABS(b.EmAngle - a.EmAngle)  as diff_em 
FROM 
mroctx as a, 
mroctx as b
WHERE
 a.EmAngle < 4
AND 
a.EmAngle < b.EmAngle 
AND
diff_em > 10 
AND
diff_em < 25
ORDER BY
ovarea desc,
diff_em desc,
left_em asc
LIMIT 1000;

"""